In [ ]:
import pandas as pd

In [ ]:
user_data = {
    'id': list(range(5)),
    'name': ['Alice', 'Brian', 'Carla', 'Devid', 'Ed']
}

In [ ]:
user_df = pd.DataFrame(user_data)
user_df

In [ ]:
tweet_data = {
    'id': list(range(100,1100,100)),
    'text': ['Hi', 'Hello', 'Yo', 'Sup', 'Bye', 'Later', 'Ciao', 'Peace', 'See ya', 'Out']
}

tweet_df = pd.DataFrame(tweet_data)
tweet_df

In [ ]:
follows_data = {
    'source': [0, 0, 1, 2, 3, 4],
    'target': [1, 2, 2, 3, 2, 2]
}

follows_df = pd.DataFrame(follows_data)
follows_df

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    

In [ ]:
from neo4j import GraphDatabase

conn = Neo4jConnection('bolt://localhost:7687', 'neo4j', 'vvanag')

In [ ]:
conn.query('DROP CONSTRAINT users_unique IF EXISTS')
conn.query('DROP CONSTRAINT tweets_unique IF EXISTS')
conn.query('CREATE CONSTRAINT users_unique on (u:User) assert u.id is UNIQUE')
conn.query('CREATE CONSTRAINT tweets_unique on (t:Tweet) assert t.id is UNIQUE')

In [ ]:
import time 
def insert_data(query, rows, batch_size=10000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {'total': total, 'batch': batch, 'time': time.time() - start}
        
    return result    

In [ ]:
def add_users(rows):
    query = """
    UNWIND $rows as row
    MERGE (:User {id: row.id, name: row.name})
    RETURN COUNT(*) as total
    """
    
    return insert_data(query, rows)

add_users(user_df)

In [ ]:
def add_tweets(rows):
    query = """
    UNWIND $rows as row
    MERGE (:Tweet {id: row.id, text: row.text})
    RETURN COUNT(*) as total
    """
    
    return insert_data(query, rows)

add_tweets(tweet_df)

In [ ]:
def add_follows(rows):
    query = """
    UNWIND $rows as row
    MATCH (source:User {id:row.source})
    MATCH (target:User {id:row.target})
    MERGE (source) -[r:FOLLOWS] -> (target)
    RETURN COUNT(r) as total
    """
    
    return insert_data(query, rows)

add_follows(follows_df)

In [ ]:
def add_writes_rel(rows):
    query = """
    UNWIND $rows as row
    MATCH (source:User {id:row.source})
    MATCH (target:Tweet {id:row.target})
    MERGE (source) -[r:WRITES] -> (target)
    RETURN COUNT(r) as total
    """
    
    return insert_data(query, rows)

tweeted_data = {
    'target': list(range(100,1100,100)),
    'source': [0 ,0, 1, 1, 2, 2, 3, 3, 4, 4]
}

tweeted_df = pd.DataFrame(tweeted_data)
tweeted_df

add_writes_rel(tweeted_df)

In [ ]:
#clean all

delete_all_edges_query = "match( ()-[r]->()) delete r;"
delete_all_nodes_query = "match (n) delete n;"

conn.query(delete_all_edges_query)
conn.query(delete_all_nodes_query)

In [ ]:
conn.query("DROP CONSTRAINT character_unique IF EXISTS")
constraint_query = "CREATE CONSTRAINT character_unique on (c:Character) assert c.id is UNIQUE"

csv_population_query = """
with 'https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-nodes.csv' as url 
load csv with headers from url as row 
merge (c:Character {id: row.Id}) set c.name=row.Label
"""

conn.query(constraint_query)
conn.query(csv_population_query)
conn.query("match (n) return count(n);")

In [ ]:
relation_query = """
with 'https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-edges.csv' as url 
load csv with headers from url as row 
match (source:Character {id: row.Source}) 
match(target:Character {id: row.Target}) 
merge (source) - [:SEASON1 {weight: toInteger(row.Weight)}] - (target)
"""

conn.query(relation_query)

In [ ]:
#clean all

delete_all_edges_query = "match( ()-[r]->()) delete r;"
delete_all_nodes_query = "match (n) delete n;"

conn.query(delete_all_edges_query)
conn.query(delete_all_nodes_query)

conn.query('match (n) return count(n)')

In [ ]:
#apoc
conn.query("DROP CONSTRAINT node_unique IF EXISTS")
constraint_query = "CREATE CONSTRAINT node_unique ON (n:Node) assert n.neo4jImportId IS UNIQUE;" #hinted by neo4j !!yay!!
conn.query(constraint_query)
conn.query('call apoc.import.json("https://raw.githubusercontent.com/cj2001/nodes2021_kg_workshop/main/json_files/wiki.json")')
conn.query('match (n) return count(n)')

In [ ]:
#and a yummy sparql query from the lesson

# select ?breweryLabel ?breweryDescription ?city ?cityLabel
# where
# {
#   ?brewery wdt:P31/wdt:P279* wd:Q131734 ;
#            wdt:P17 wd:Q183 ;
#            wdt:P131 ?city .
#   service wikibase:label {
#     bd:serviceParam wikibase:language "en, de" .
#   }
# }
# order by asc(?cityLabel)